In [0]:
"""
Central configuration for the traffic ETL pipeline.

This module defines a single ``Config`` class that centralizes all
catalog names, database names and storage locations used in the ETL
process.  The ``conf`` object is provided as a convenient singleton
instance.
"""
import os

class Config:
    def __init__(self, env=None, storage_account=None, metastore_account=None):
        # 1) resolve environment (widgets or env vars can set these before import)
        self.env = (env or os.getenv("ENV") or "dev").strip().lower()
        sa = (storage_account   or os.getenv("STORAGE_ACCOUNT")   or "trafficsa2").strip()
        ms = (metastore_account or os.getenv("METASTORE_ACCOUNT") or sa).strip()

        # 2) UC names
        self.catalog = self.env
        self.db_name = "traffic_db"

        # 3) FQDNs
        self.sa_fqdn = f"{sa}.dfs.core.windows.net"
        self.ms_fqdn = f"{ms}.dfs.core.windows.net"

        # 4) storage locations
        self.metastore_root  = f"abfss://traffic-metastore-root@{self.ms_fqdn}/"
        self.managed_data    = f"abfss://traffic-managed-{self.env}@{self.sa_fqdn}/"
        self.unmanaged_data  = f"abfss://traffic-unmanaged-{self.env}@{self.sa_fqdn}/"

        # 5) base paths
        self.raw_data_path   = f"{self.unmanaged_data}data_zone"
        self.checkpoint_base = f"{self.unmanaged_data}checkpoint_zone"

        # 6) logical table names (canonical)
        self.bronze_table  = "raw_traffic"
        self.silver_table  = "traffic_silver_events"   # canonical (plural)
        self.region_lookup = "region_lookup"

    def table_fqn(self, table: str) -> str:
        return f"{self.catalog}.{self.db_name}.{table}"

# default instance; will use ENV/STORAGE_ACCOUNT/METASTORE_ACCOUNT if set
conf = Config()
